In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation

/Users/Roman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("../credit_train.csv", sep = ";", encoding='cp1251')
test = pd.read_csv("../credit_test.csv", sep = ";", encoding='cp1251')

In [3]:
train.head(2)

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,"59998,00",10,1.6,"0,770249",GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,28,MAR,UMN,"10889,00",6,1.1,"0,248514",GRD,МОСКВА,43000.0,2.0,0.0,0


In [4]:
y = train['open_account_flg']

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 15 columns):
client_id               170746 non-null int64
gender                  170746 non-null object
age                     170746 non-null int64
marital_status          170746 non-null object
job_position            170746 non-null object
credit_sum              170746 non-null object
credit_month            170746 non-null int64
tariff_id               170746 non-null float64
score_shk               170746 non-null object
education               170746 non-null object
living_region           170554 non-null object
monthly_income          170745 non-null float64
credit_count            161516 non-null float64
overdue_credit_count    161516 non-null float64
open_account_flg        170746 non-null int64
dtypes: float64(4), int64(4), object(7)
memory usage: 19.5+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91940 entries, 0 to 91939
Data columns (total 14 columns):
client_id               91940 non-null int64
gender                  91940 non-null object
age                     91940 non-null int64
marital_status          91940 non-null object
job_position            91940 non-null object
credit_sum              91940 non-null object
credit_month            91940 non-null int64
tariff_id               91940 non-null float64
score_shk               91940 non-null object
education               91940 non-null object
living_region           91824 non-null object
monthly_income          91940 non-null int64
credit_count            87237 non-null float64
overdue_credit_count    87237 non-null float64
dtypes: float64(3), int64(4), object(7)
memory usage: 9.8+ MB


I работа с пропусками в данных

In [8]:
train[train.monthly_income.isnull()]

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
19639,19640,F,33,MAR,BIS,"13009,00",10,1.6,"0,535834",SCH,ПРИМОРСКИЙ КРАЙ,NaN,2.0,0.0,0


In [9]:
train.monthly_income.fillna(value=train.monthly_income.mean(), inplace=True)

In [10]:
train.living_region.fillna(value="Unknown", inplace=True)
test.living_region.fillna(value="Unknown", inplace=True)

In [11]:
train[train.credit_count.isnull()]

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
25,26,F,22,UNM,SPC,"30000,00",18,1.10,"0,550770",UGR,РЕСП. БАШКОРТОСТАН,30000.0,NaN,NaN,1
50,51,F,35,MAR,SPC,"13779,00",10,1.60,"0,677837",GRD,ОБЛ НОВОСИБИРСКАЯ,28000.0,NaN,NaN,0
52,53,F,33,DIV,DIR,"15379,20",10,1.22,"0,566717",GRD,САНКТ-ПЕТЕРБУРГ,60000.0,NaN,NaN,0
72,73,M,21,UNM,ATP,"14309,00",10,1.10,"0,293619",SCH,МОСКВА,30000.0,NaN,NaN,1
74,75,F,27,MAR,SPC,"15248,00",10,1.60,"0,591578",SCH,ОРЕНБУРГСКАЯ ОБЛ,10000.0,NaN,NaN,0
87,88,F,25,MAR,SPC,"20683,00",10,1.32,"0,504309",SCH,РОСТОВСКАЯ ОБЛ,30000.0,NaN,NaN,1
94,95,M,26,UNM,SPC,"26088,00",10,1.60,"0,756208",GRD,КРАСНОЯРСКИЙ КРАЙ,25000.0,NaN,NaN,0
138,139,M,48,MAR,SPC,"10159,00",10,1.60,"0,661510",SCH,СТАВРОПОЛЬСКИЙ КРАЙ,25500.0,NaN,NaN,0
153,154,F,23,UNM,SPC,"30431,11",10,1.60,"0,725202",GRD,ОБЛ БЕЛГОРОДСКАЯ,15000.0,NaN,NaN,0
154,155,F,53,MAR,SPC,"7379,00",6,1.40,"0,569212",UGR,ОБЛ ПСКОВСКАЯ,57000.0,NaN,NaN,0


In [12]:
train.credit_count.fillna(value = train.credit_count.mean(), inplace=True)
train.overdue_credit_count.fillna(value = train.overdue_credit_count.mean(), inplace=True)

In [13]:
test.credit_count.fillna(value = train.credit_count.mean(), inplace=True)
test.overdue_credit_count.fillna(value = train.overdue_credit_count.mean(), inplace=True)

In [14]:
train.gender = train.gender.map({"M":1, "F":0})
test.gender = test.gender.map({"M":1, "F":0})

In [15]:
train.rename(columns={"open_account_flg":"target"}, inplace=True)

II выделим группы людей по наличию высшего образования, работе и пенсии 

In [16]:
### по образованию
train["high_edu"] = ((train["education"] != "SCH") & (train["education"] != "UGR"))

### по работе
# 1. не работающие граждание
train["not_work"] = ((train.job_position == "HSK") | (train.job_position == "INV") 
                     | (train.job_position == "NOR") | (train.job_position == "PNS"))

# 2. обычные работники
train["usual_workers"] = ((train.job_position == "SPC") | (train.job_position == "WOI") 
                     | (train.job_position == "WRK") | (train.job_position == "ATP") 
                     | (train.job_position == "WRP"))

# 3. начальники
train["head_workers"] = ((train.job_position == "DIR") | (train.job_position == "UMN")
                         | (train.job_position == "BIS") | (train.job_position == "INP"))

# 4. имеют собственный бизнес
train["have_business"] = ((train.job_position == "BIS") | (train.job_position == "INP"))

### по браку
train["single"] = ((train.marital_status == "UNM") | (train.marital_status == "DIV")
                   | (train.marital_status == "WID"))

In [17]:
### по образованию
test["high_edu"] = ((test["education"] != "SCH") & (test["education"] != "UGR"))

### по работе
# 1. не работающие граждание
test["not_work"] = ((test.job_position == "HSK") | (test.job_position == "INV") 
                     | (test.job_position == "NOR") | (test.job_position == "PNS"))

# 2. обычные работники
test["usual_workers"] = ((test.job_position == "SPC") | (test.job_position == "WOI") 
                     | (test.job_position == "WRK") | (test.job_position == "ATP") 
                     | (test.job_position == "WRP"))

# 3. начальники
test["head_workers"] = ((test.job_position == "DIR") | (test.job_position == "UMN")
                         | (test.job_position == "BIS") | (test.job_position == "INP"))

# 4. имеют собственный бизнес
test["have_business"] = ((test.job_position == "BIS") | (test.job_position == "INP"))

### по браку
test["single"] = ((test.marital_status == "UNM") | (test.marital_status == "DIV")
                   | (test.marital_status == "WID"))

III кодировка категориальных признаков, кроме living region

In [18]:
train_col = train.columns.drop("target")

In [19]:
dummy_df = pd.concat([train[train_col], test], axis=0)

In [20]:
dummy_df = pd.get_dummies(dummy_df, columns=["education","job_position", "marital_status"], drop_first=True)

In [21]:
train = dummy_df[:train.shape[0]]
test = dummy_df[train.shape[0]:]

In [22]:
train.columns

Index(['client_id', 'gender', 'age', 'credit_sum', 'credit_month', 'tariff_id',
       'score_shk', 'living_region', 'monthly_income', 'credit_count',
       'overdue_credit_count', 'high_edu', 'not_work', 'usual_workers',
       'head_workers', 'have_business', 'single', 'education_GRD',
       'education_PGR', 'education_SCH', 'education_UGR', 'job_position_BIS',
       'job_position_BIU', 'job_position_DIR', 'job_position_HSK',
       'job_position_INP', 'job_position_INV', 'job_position_NOR',
       'job_position_ONB', 'job_position_PNA', 'job_position_PNI',
       'job_position_PNS', 'job_position_PNV', 'job_position_SPC',
       'job_position_UMN', 'job_position_WOI', 'job_position_WRK',
       'job_position_WRP', 'marital_status_DIV', 'marital_status_MAR',
       'marital_status_UNM', 'marital_status_WID'],
      dtype='object')

IV кодировка региона через среднюю зарплату

In [23]:
def code_mean(data, cat_feature, real_feature):
    return (data[cat_feature].map(data.groupby(cat_feature)[real_feature].mean()))

In [24]:
train['monthly_income'].astype(float)
test['monthly_income'].astype(float)

train["city_mean_income"] = code_mean(train, 'living_region', 'monthly_income')
train.city_mean_income.fillna(value=train.city_mean_income.mean(), inplace=True)
train.drop("living_region", axis=1, inplace=True)

test["city_mean_income"] = code_mean(test, 'living_region', 'monthly_income')
test.city_mean_income.fillna(value=train.city_mean_income.mean(), inplace=True)
test.drop("living_region", axis=1, inplace=True)

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Roman/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Roman/anaconda/lib/python3.6/sit

V Генерация данных о зарплате и выплатам по кредиту

In [27]:
tmp = train['credit_sum'].apply(lambda x: x.replace(",","."))
train['credit_sum'] = tmp
tmp = test['credit_sum'].apply(lambda x: x.replace(",","."))
test['credit_sum'] = tmp

train['credit_sum'] = train['credit_sum'].astype(float)
test['credit_sum'] = test['credit_sum'].astype(float)

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:

credit_pay - ежемесячный расход на кредит

money_for_life - разность между зп и платой по кредиту

dif_city - разница между зп и средней зп по региону

In [28]:
train["credit_pay"] = train["credit_sum"] / train["credit_month"]
train["money_for_life"] = train["monthly_income"] - train["credit_pay"]
train["dif_city"] = train["monthly_income"] - train["city_mean_income"]

test["credit_pay"] = test["credit_sum"] / test["credit_month"]
test["money_for_life"] = test["monthly_income"] - test["credit_pay"]
test["dif_city"] = test["monthly_income"] - test["city_mean_income"]

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

VI анализ score_shk

In [29]:
tmp = train['score_shk'].apply(lambda x: x.replace(",","."))
train.score_shk = tmp

tmp = test['score_shk'].apply(lambda x: x.replace(",","."))
test.score_shk = tmp

train['score_shk'] = train['score_shk'].astype(float)
test['score_shk'] = test['score_shk'].astype(float)

/Users/Roman/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [30]:
train['target'] = y

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170746 entries, 0 to 170745
Data columns (total 46 columns):
client_id               170746 non-null int64
gender                  170746 non-null int64
age                     170746 non-null int64
credit_sum              170746 non-null float64
credit_month            170746 non-null int64
tariff_id               170746 non-null float64
score_shk               170746 non-null float64
monthly_income          170746 non-null float64
credit_count            170746 non-null float64
overdue_credit_count    170746 non-null float64
high_edu                170746 non-null bool
not_work                170746 non-null bool
usual_workers           170746 non-null bool
head_workers            170746 non-null bool
have_business           170746 non-null bool
single                  170746 non-null bool
education_GRD           170746 non-null uint8
education_PGR           170746 non-null uint8
education_SCH           170746 non-null uint8
education

In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91940 entries, 0 to 91939
Data columns (total 45 columns):
client_id               91940 non-null int64
gender                  91940 non-null int64
age                     91940 non-null int64
credit_sum              91940 non-null float64
credit_month            91940 non-null int64
tariff_id               91940 non-null float64
score_shk               91940 non-null float64
monthly_income          91940 non-null float64
credit_count            91940 non-null float64
overdue_credit_count    91940 non-null float64
high_edu                91940 non-null bool
not_work                91940 non-null bool
usual_workers           91940 non-null bool
head_workers            91940 non-null bool
have_business           91940 non-null bool
single                  91940 non-null bool
education_GRD           91940 non-null uint8
education_PGR           91940 non-null uint8
education_SCH           91940 non-null uint8
education_UGR           91940 

In [ ]:
train.to_csv("../datasets/train_without_oversampling.csv", columns=train.columns, index=False)
test.to_csv("../datasets/test_without_oversampling.csv", columns=test.columns, index=False)